In [ ]:
import glob

import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import cv2

In [ ]:
images = {
    int(path[17:-4]): np.load(path).reshape((720, 1280, 3))
    for path in glob.glob("data/photo_focal_*.npy")
}
print(len(images))
print(images.keys())

In [ ]:
# canny_images = [
#     cv2.Canny(cv2.blur(img, (3, 3)), 10, 70, apertureSize=3) for img in images.values()
# ]
px.imshow(np.array(images), animation_frame=0, width=5_00).show()

In [ ]:
px.imshow(images[4600]).show()

In [ ]:
%timeit cv2.cvtColor(images[4600], cv2.COLOR_BGR2GRAY)

In [ ]:
arucoDict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_6X6_250)
arucoParams = cv2.aruco.DetectorParameters()
(corners, ids, rejected) = cv2.aruco.detectMarkers(
    images[4600][::4, ::4], arucoDict, parameters=arucoParams
)
px.imshow(
    cv2.aruco.drawDetectedMarkers(images[4600][::4, ::4, ::-1].copy(), corners, ids)
).show()

In [ ]:
# 14.3, 14.4 cm shape
marker_size_mm = 144


def guess_focal(img: np.array, dist_mm: int, size_mm: int = marker_size_mm):
    # downsample = 4

    # img = cv2.cvtColor(img[::downsample, ::downsample], cv2.COLOR_BGR2GRAY)
    check = cv2.aruco.detectMarkers(img, arucoDict, parameters=arucoParams)[0]
    if not check:
        print(f"Warning: distance {dist_mm} can't be identified")
        return None
    cs = check[0][0]
    pixel_size = np.mean(
        [
            np.sqrt(np.square((cs[0] - cs[3]) * downsample).sum()),
            np.sqrt(np.square((cs[1] - cs[2]) * downsample).sum()),
        ]
    )
    return pixel_size * dist_mm / size_mm


%timeit guess_focal(images[4700], dist_mm=4700)

In [ ]:
focals = []
for dist, im in images.items():
    f = guess_focal(im, dist)
    if f is None:
        continue
    focals.append(f)
    print(f"{dist: >7} & {focals[-1]: >10.2f} \\\\")
avg_focal = np.mean(focals)
print(f"\nAverage: {avg_focal}")

In [ ]:
print(focals)
np.std(focals)

In [ ]:
test_cam_mat = np.array(
    [
        [avg_focal, 0, 0],
        [0, avg_focal, 0],
        [0, 0, 1],
    ]
)

In [ ]:
images[500]

In [ ]:
_object_points = np.array(
    [
        [-marker_size_mm / 2, marker_size_mm / 2, 0],
        [marker_size_mm / 2, marker_size_mm / 2, 0],
        [marker_size_mm / 2, -marker_size_mm / 2, 0],
        [-marker_size_mm / 2, -marker_size_mm / 2, 0],
    ]
)


def estimate_pose(cn, cam_internal=test_cam_mat, obj_points=_object_points):
    ret, r_vec, t_vec = cv2.solvePnP(
        obj_points,
        imagePoints=cn,
        cameraMatrix=cam_internal,
        distCoeffs=np.zeros((4,)),
        flags=cv2.SOLVEPNP_IPPE_SQUARE,
    )
    assert ret, "Failed to solve PnP for pose estimation!"
    r_mat = np.empty((3, 3))
    cv2.Rodrigues(r_vec, dst=r_mat)
    return r_mat, t_vec


estimate_pose(corners[0][0])

In [ ]:
cam_test = np.array([np.load(f"data/cam_rotate_{nth}.npy") for nth in range(20)])
len(cam_test)

In [ ]:
(list(images.values()))[0].shape

In [ ]:
px.imshow(np.array(list(images.values())), animation_frame=0).show()